# Hot Chocolate

Figuring out the qutrit handling

In [1]:
# All Imports

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

import qiskit
from qiskit import pulse

from qiskit_dynamics import Solver, DynamicsBackend
from qiskit_dynamics.pulse import InstructionToSignals
from qiskit_dynamics.array import Array

from qiskit.quantum_info import Statevector, DensityMatrix, Operator, SparsePauliOp
from qiskit.circuit.parameter import Parameter

import jax
import jax.numpy as jnp
from jax import jit, vmap, block_until_ready, config

import chex

from typing import Optional, Union

Array.set_default_backend('jax')
config.update('jax_enable_x64', True)
config.update('jax_platform_name', 'cpu')

In [2]:
# Constructing a Two Qutrit Hamiltonian

dim = 3

v0 = 4.86e9
anharm0 = -0.32e9
r0 = 0.22e9

v1 = 4.97e9
anharm1 = -0.32e9
r1 = 0.26e9

J = 0.002e9

a = np.diag(np.sqrt(np.arange(1, dim)), 1)
adag = np.diag(np.sqrt(np.arange(1, dim)), -1)
N = np.diag(np.arange(dim))

ident = np.eye(dim, dtype=complex)
full_ident = np.eye(dim**2, dtype=complex)

N0 = np.kron(ident, N)
N1 = np.kron(N, ident)

a0 = np.kron(ident, a)
a1 = np.kron(a, ident)

a0dag = np.kron(ident, adag)
a1dag = np.kron(adag, ident)


static_ham0 = 2 * np.pi * v0 * N0 + np.pi * anharm0 * N0 * (N0 - full_ident)
static_ham1 = 2 * np.pi * v1 * N1 + np.pi * anharm1 * N1 * (N1 - full_ident)

static_ham_full = static_ham0 + static_ham1 + 2 * np.pi * J * ((a0 + a0dag) @ (a1 + a1dag))

drive_op0 = 2 * np.pi * r0 * (a0 + a0dag)
drive_op1 = 2 * np.pi * r1 * (a1 + a1dag)

In [3]:
batchsize = 400

amp_vals = jnp.linspace(0.5, 0.99, batchsize, dtype=jnp.float64).reshape(-1, 1)
sigma_vals = jnp.linspace(20, 80, batchsize, dtype=jnp.int8).reshape(-1, 1)
freq_vals = jnp.linspace(-0.5, 0.5, batchsize, dtype=jnp.float64).reshape(-1, 1) * 1e6
batch_params = jnp.concatenate((amp_vals, sigma_vals, freq_vals), axis=-1)

batch_y0 = jnp.tile(np.ones(9), (batchsize, 1))
batch_obs = jnp.tile(N0, (batchsize, 1, 1))

print(f"Batched Params Shape: {batch_params.shape}")

# Constructing a custom function that takes as input a parameter vector and returns the simulated state

def standard_func(params):
    amp, sigma, freq = params

    # Here we use a Drag Pulse as defined in qiskit pulse as its already a Scalable Symbolic Pulse
    special_pulse = pulse.Drag(
        duration=320,
        amp=amp,
        sigma=sigma,
        beta=0.1,
        angle=0.1,
        limit_amplitude=False
    )

    with pulse.build(default_alignment='sequential') as sched:
        d0 = pulse.DriveChannel(0)
        d1 = pulse.DriveChannel(1)
        u0 = pulse.ControlChannel(0)
        u1 = pulse.ControlChannel(1)

        pulse.shift_frequency(freq, d0)
        pulse.play(special_pulse, d0)

        pulse.shift_frequency(freq, d1)
        pulse.play(special_pulse, d1)

        pulse.shift_frequency(freq, u0)
        pulse.play(special_pulse, u0)

        pulse.shift_frequency(freq, u1)
        pulse.play(special_pulse, u1)
    
    return sched

Batched Params Shape: (400, 3)


In [4]:
# Constructing the new solver

dt = 1/4.5e9
atol = 1e-2
rtol = 1e-4

t_linspace = np.linspace(0.0, 400e-9, 11)
t_span = np.array([t_linspace[0], t_linspace[-1]])

ham_ops = [drive_op0, drive_op1, drive_op0, drive_op1]
ham_chans = ["d0", "d1", "u0", "u1"]
chan_freqs = {"d0": v0, "d1": v1, "u0": v1, "u1": v0}

solver = Solver(
    static_hamiltonian=static_ham_full,
    hamiltonian_operators=ham_ops,
    rotating_frame=static_ham_full,
    hamiltonian_channels=ham_chans,
    channel_carrier_freqs=chan_freqs,
    dt=dt,
)

class JaxifiedSolver:
    def __init__(
        self,
        schedule_func,
        dt,
        carrier_freqs,
        ham_chans,
        t_span,
        rtol,
        atol
    ):
        super().__init__()
        self.schedule_func = schedule_func
        self.dt = dt
        self.carrier_freqs = carrier_freqs
        self.ham_chans = ham_chans
        self.t_span = t_span
        self.rtol = rtol
        self.atol = atol
        self.fast_batched_sim = jit(vmap(self.run_sim))

    def run_sim(self, y0, obs, params):
        sched = self.schedule_func(params)

        converter = InstructionToSignals(self.dt, carriers=self.carrier_freqs, channels=self.ham_chans)

        signals = converter.get_signals(sched)

        results = solver.solve(
            t_span=self.t_span,
            y0=y0 / jnp.linalg.norm(y0),
            t_eval=self.t_span,
            signals=signals,
            rtol=self.rtol,
            atol=self.atol,
            convert_results=False,
            method='jax_odeint'
        )

        state_vec = results.y.data[-1]
        state_vec = state_vec / jnp.linalg.norm(state_vec)
        two_vec = state_vec[:4]
        evolved_vec = jnp.dot(obs, two_vec)
        new_vec = jnp.concatenate((evolved_vec, state_vec[4:]))
        probs_vec = jnp.abs(new_vec)**2
        probs_vec = jnp.clip(probs_vec, a_min=0.0, a_max=1.0)

        # Shots instead of probabilities

        return probs_vec
    
    def estimate(self, batch_y0, batch_obs, batch_params):
        ops_mat = [b.to_matrix() for b in batch_obs]
        ops_arr = jnp.array(ops_mat)
        return self.fast_batched_sim(batch_y0, ops_arr, batch_params)



In [5]:
j_solver = JaxifiedSolver(
    schedule_func=standard_func,
    dt=dt,
    carrier_freqs=chan_freqs,
    ham_chans=ham_chans,
    t_span=t_span,
    rtol=rtol,
    atol=atol
)

In [6]:
ops_list = [SparsePauliOp(["IX"]), SparsePauliOp(["IY"]), SparsePauliOp(["YZ"]), SparsePauliOp(["ZX"])] * 100

batch_res = j_solver.estimate(
    batch_y0,
    ops_list,
    batch_params
)

In [7]:
%timeit j_solver.estimate(batch_y0,ops_list,batch_params)

7.03 s ± 768 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
op_x = SparsePauliOp('IXXYI')
arr = op_x.to_matrix()
print(arr.shape)
print(op_x.to_matrix())

(32, 32)
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]


In [22]:
dim_size = 8

qudit = jnp.array([1.0, 0.0, 0.0, 0.0])
qubit_op = jnp.array([[0., 1.0], [1.0, 0.0]])
big_ident = jnp.identity(dim_size)
big_op = big_ident.at[:2,:2].set(qubit_op)
big_op

Array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float64)

In [76]:
op_str = "XI"
num_qubits = len(op_str)
qudit_dim_size = 3
init_state = np.zeros(qudit_dim_size ** num_qubits, dtype=np.complex64)
init_state[1] = 1

base_gates_dict = {
    "I": jnp.array([[1.0, 0.], [0., 1.]]),
    "X": jnp.array([[0., 1.], [1., 0.]]),
    "Y": jnp.array([[0., -1.0j], [1.0j, 0.]]),
    "Z": jnp.array([[1., 0.], [0., -1.]])
}

def PauliToQuditMatrix(inp_str: str, qudit_dim_size: Optional[int] = 4):
    word_list = list(inp_str)
    qudit_op_list = []
    for word in word_list:
        qubit_op = base_gates_dict[word]
        qud_op = np.identity(qudit_dim_size, dtype=np.complex64)
        qud_op[:2,:2] = qubit_op
        qudit_op_list.append(qud_op)
    complete_op = qudit_op_list[0]
    for i in range(1,len(qudit_op_list)):
        complete_op = np.kron(complete_op, qudit_op_list[i])
    return complete_op

def evolve_state(batch_state, batch_var_str):
    return_state = []
    for state, var_str in zip(batch_state, batch_var_str):
        complete_op = PauliToQuditMatrix(var_str, qudit_dim_size)
        return_state.append(complete_op @ state)
    return return_state

b_size = 400

batch_state = [init_state] * b_size
batch_var_str = [op_str] * b_size

%timeit evolve_state(batch_state, batch_var_str)

44.5 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
class JaxedSolver:
    def __init__(
        self,
        schedule_func,
        dt,
        carrier_freqs,
        ham_chans,
        t_span,
        rtol,
        atol
    ):
        super().__init__()
        self.schedule_func = schedule_func
        self.dt = dt
        self.carrier_freqs = carrier_freqs
        self.ham_chans = ham_chans
        self.t_span = t_span
        self.rtol = rtol
        self.atol = atol
        self.fast_batched_sim = jit(vmap(self.run_sim))

    def run_sim(self, y0, obs, params):
        sched = self.schedule_func(params)

        converter = InstructionToSignals(self.dt, carriers=self.carrier_freqs, channels=self.ham_chans)

        signals = converter.get_signals(sched)

        results = solver.solve(
            t_span=self.t_span,
            y0=y0 / jnp.linalg.norm(y0),
            t_eval=self.t_span,
            signals=signals,
            rtol=self.rtol,
            atol=self.atol,
            convert_results=False,
            method='jax_odeint'
        )

        state_vec = results.y.data[-1]
        state_vec = state_vec / jnp.linalg.norm(state_vec)
        new_vec = obs @ state_vec
        probs_vec = jnp.abs(new_vec)**2
        probs_vec = jnp.clip(probs_vec, a_min=0.0, a_max=1.0)

        # Shots instead of probabilities

        return probs_vec
    
    def estimate2(self, batch_y0, batch_params, batch_obs_str):
        batch_obs = jnp.zeros((batch_y0.shape[0], batch_y0.shape[1], batch_y0.shape[1]), dtype=jnp.complex64)
        for i, b_str in enumerate(batch_obs_str):
            batch_obs = batch_obs.at[i].set(PauliToQuditMatrix(b_str, dim))
        return self.fast_batched_sim(batch_y0, batch_obs, batch_params)

In [103]:
j_solver_2 = JaxedSolver(
    schedule_func=standard_func,
    dt=dt,
    carrier_freqs=chan_freqs,
    ham_chans=ham_chans,
    t_span=t_span,
    rtol=rtol,
    atol=atol
)

In [104]:
ops_str_list = ["IX", "XY", "ZX", "ZI"] * 100

batch_res = j_solver_2.estimate2(
    batch_y0,
    batch_params,
    ops_str_list
)

In [107]:
%timeit  j_solver_2.estimate2(batch_y0, batch_params, ops_str_list)

7.21 s ± 73.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
